This notebook documents the code used for the automatic detection of ICMEs in the full dataset with combined training. It is running on Python 3.8.5 with dependencies listed in the requirements.txt file.

In [1]:
#only run if GPU available

#Set devices_id
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-07-25 22:37:13.252322: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


True

2022-07-25 22:37:14.524785: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 15405 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:01:00.0, compute capability: 6.0


In [2]:
# Don't print warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pds
import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope

In [3]:
import event as evt

# load ICME catalog data

[ic,header,parameters] = pickle.load(open('data/HELCATS_ICMECAT_v20_pandas.p', "rb" ))


# extract important values

isc = ic.loc[:,'sc_insitu'] 
starttime = ic.loc[:,'icme_start_time']
endtime = ic.loc[:,'mo_end_time']

# Event indices from Wind, STEREO A and STEREO B

iwinind = np.where(isc == 'Wind')[0]
istaind = np.where(isc == 'STEREO-A')[0]
istbind = np.where(isc == 'STEREO-B')[0]

winbegin = starttime[iwinind]
winend = endtime[iwinind]

stabegin = starttime[istaind]
staend = endtime[istaind]

stbbegin = starttime[istbind]
stbend = endtime[istbind]

# get list of events

evtListw = evt.read_cat(winbegin, winend, iwinind)
evtLista = evt.read_cat(stabegin, staend, istaind)
evtListb = evt.read_cat(stbbegin, stbend, istbind)

# Load Wind data
[win, winheader] = pickle.load(open("data/wind_2007_2021_heeq_ndarray.p", "rb"))

# Load STEREO-A data
[sta, atta] = pickle.load(open("data/stereoa_2007_2021_sceq_ndarray.p", "rb"))

# Load STEREO-B data
[stb, attb, bheader] = pickle.load(open("data/stereob_2007_2014_sceq_ndarray.p", "rb"))

In [4]:
import features

# pre process on the WIND data set

datawin = pds.DataFrame(win)
datawin['time'] = matplotlib.dates.num2date(datawin['time'], tz=None) 
datawin['time'] = pds.to_datetime(datawin['time'], format="%Y/%m/%d %H:%M")
datawin.set_index('time',  inplace=True)
datawin.index.name = None
datawin.index = datawin.index.tz_localize(None)
datawin.drop(['x', 'y', 'z', 'r', 'lat', 'lon'], axis = 1,  inplace=True)

# compute additional features

features.computeBetawiki(datawin)
features.computePdyn(datawin)
features.computeTexrat(datawin)

# resample data
datawin = datawin.resample('10T').mean().dropna()

# pre process on the STEREO A data set

dataa = pds.DataFrame(sta)
dataa['time'] = matplotlib.dates.num2date(dataa['time'], tz=None) 
dataa['time'] = pds.to_datetime(dataa['time'], format="%Y/%m/%d %H:%M")
dataa.set_index('time',  inplace=True)
dataa.index.name = None
dataa.index = dataa.index.tz_localize(None)
dataa.drop(['x', 'y', 'z', 'r', 'lat', 'lon'], axis = 1,  inplace=True)

# compute additional features

features.computeBetawiki(dataa)
features.computePdyn(dataa)
features.computeTexrat(dataa)

# resample data
dataa = dataa.resample('10T').mean().dropna()

# pre process on the STEREO B data set

datab = pds.DataFrame(stb)
datab['time'] = matplotlib.dates.num2date(datab['time'], tz=None) 
datab['time'] = pds.to_datetime(datab['time'], format="%Y/%m/%d %H:%M")
datab.set_index('time',  inplace=True)
datab.index.name = None
datab.index = datab.index.tz_localize(None)
datab.drop(['x', 'y', 'z', 'r', 'lat', 'lon'], axis = 1,  inplace=True)

# compute additional features

features.computeBetawiki(datab)
features.computePdyn(datab)
features.computeTexrat(datab)

# resample data
datab = datab.resample('10T').mean().dropna()

In [5]:
# delete empty events

evtListw = evt.clearempties(evtListw,datawin)
evtLista = evt.clearempties(evtLista,dataa)
evtListb = evt.clearempties(evtListb,datab)

In [6]:
# perform scaling on WIND

scale = StandardScaler()
scale.fit(datawin)

wind_scaled = pds.DataFrame(index = datawin.index, columns = datawin.columns, data = scale.transform(datawin))

# perform scaling on STEREO A

scale = StandardScaler()
scale.fit(dataa)

sta_scaled = pds.DataFrame(index = dataa.index, columns = dataa.columns, data = scale.transform(dataa))

# perform scaling on STEREO B

scale = StandardScaler()
scale.fit(datab)

stb_scaled = pds.DataFrame(index = datab.index, columns = datab.columns, data = scale.transform(datab))

In [7]:
import preprocess

truelabelw = pds.DataFrame(preprocess.get_truelabel(wind_scaled, evtListw))
truelabela = pds.DataFrame(preprocess.get_truelabel(sta_scaled, evtLista))
truelabelb = pds.DataFrame(preprocess.get_truelabel(stb_scaled, evtListb))

In [8]:
split = 2
testw, valw, trainw = preprocess.getbalancedsplit(split, 'wind') 
testa, vala, traina = preprocess.getbalancedsplit(split, 'stereoa') 
testb, valb, trainb = preprocess.getbalancedsplit(split, 'stereob') 

X_testw, Y_testw, X_valw, Y_valw, X_trainw, Y_trainw = preprocess.getdatas(trainw,testw,valw,wind_scaled,truelabelw)
X_testa, Y_testa, X_vala, Y_vala, X_traina, Y_traina = preprocess.getdatas(traina,testa,vala,sta_scaled,truelabela)
X_testb, Y_testb, X_valb, Y_valb, X_trainb, Y_trainb = preprocess.getdatas(trainb,testb,valb,stb_scaled,truelabelb)


print(preprocess.printpercentage(Y_testw))
print(preprocess.printpercentage(Y_trainw))
print(preprocess.printpercentage(Y_valw))

label    0.043376
dtype: float64
label    0.044865
dtype: float64
label    0.056375
dtype: float64


To train the model on all datasets at once, we need to join them.

In [9]:
X_train = X_trainw.append(X_traina, sort=False)
X_train = X_train.append(X_trainb, sort = False)

Y_train = Y_trainw.append(Y_traina, sort=False)
Y_train = Y_train.append(Y_trainb, sort = False)

####

X_val = X_valw.append(X_vala, sort=False)
X_val = X_val.append(X_valb, sort = False)

Y_val = Y_valw.append(Y_vala, sort=False)
Y_val = Y_val.append(Y_valb, sort = False)

####

X_test = X_testw.append(X_testa, sort=False)
X_test = X_test.append(X_testb, sort = False)

Y_test = Y_testw.append(Y_testa, sort=False)
Y_test = Y_test.append(Y_testb, sort = False)

In [10]:
from metrics import dice_coef, dice_loss, true_skill_score
from unetgen import UnetGen
from cycliclr import *

model_path = "model" + str(split)

## Parameters

C = 10 # number of channels
t = 1024 # window size
image_size = (t,1,C)
batch_size = 32
epochs =500
expclr = CyclicLR(base_lr=0.00001, max_lr=0.01, step_size=1000)

## Generator
train_gen = UnetGen(X_train, Y_train, length=int(t), stride = 120, shuffle = True, batch_size=batch_size)
valid_gen = UnetGen(X_val, Y_val, length=int(t), stride = 120,shuffle = True,batch_size=batch_size)

In [11]:
from model import ResUnetPlusPlus

#### ResUnet++

arch = ResUnetPlusPlus(input_shape=image_size)
model = arch.build_model()

optimizer = Adam()

metrics = [Recall(), Precision(), dice_coef,true_skill_score, MeanIoU(num_classes=2)]
model.compile(loss=dice_loss, optimizer=optimizer, metrics=metrics) 

checkpoint = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
callbacks = [checkpoint, early_stopping,expclr]


2022-07-25 22:39:03.579874: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15405 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:01:00.0, compute capability: 6.0


In [12]:
#### uncomment to train
#model.fit(train_gen,validation_data=valid_gen,epochs=epochs,callbacks=callbacks)

After the training process is finished, we continue by testing our model on unseen data:

In [13]:
import postprocess
from glob import glob

batch_size = 1


## Model
with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef, 'true_skill_score': true_skill_score}):
    model = load_model(model_path)

# TEST WIND

test_genw = UnetGen(X_testw, Y_testw, length=int(t), stride = int(t),batch_size=batch_size)

model.evaluate(test_genw, verbose=1)

resultw = postprocess.generate_result(X_testw, Y_testw, model,C,t)

# TEST STEREO A

test_gena = UnetGen(X_testa, Y_testa, length=int(t), stride = int(t),batch_size=batch_size)

model.evaluate(test_gena, verbose=1)

resulta = postprocess.generate_result(X_testa, Y_testa, model,C,t)

# TEST STEREO B

test_genb = UnetGen(X_testb, Y_testb, length=int(t), stride = int(t),batch_size=batch_size)

model.evaluate(test_genb, verbose=1)

resultb = postprocess.generate_result(X_testb, Y_testb, model,C,t)

2022-07-25 22:39:10.875028: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-25 22:39:12.752435: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8200


149/149 [==============================] - 7s 24ms/step - loss: 0.2153 - recall: 0.4385 - precision: 0.7554 - dice_coef: 0.7847 - true_skill_score: 0.8198 - mean_io_u: 0.6646


100%|█████████████████████████████████████████| 149/149 [00:15<00:00,  9.85it/s]


175/175 [==============================] - 4s 24ms/step - loss: 0.2247 - recall: 0.6328 - precision: 0.6005 - dice_coef: 0.7753 - true_skill_score: 0.8907 - mean_io_u: 0.7076


100%|█████████████████████████████████████████| 175/175 [00:16<00:00, 10.40it/s]


101/101 [==============================] - 2s 24ms/step - loss: 0.3104 - recall: 0.4407 - precision: 0.6765 - dice_coef: 0.6896 - true_skill_score: 0.7573 - mean_io_u: 0.6499


100%|█████████████████████████████████████████| 101/101 [00:09<00:00, 10.52it/s]


In [14]:
# evaluate WIND

resultw.index = pds.to_datetime(resultw.index)
resultbinw = postprocess.make_binary(resultw['pred'], 0.5)
eventsw = postprocess.makeEventList(resultbinw, 1, 10)
ICMEsw = postprocess.removeCreepy(eventsw, 3)
test_cloudsw = [x for x in evtListw if (x.begin.year in testw)]

# evaluate STEREO A

resulta.index = pds.to_datetime(resulta.index)
resultbina = postprocess.make_binary(resulta['pred'], 0.5)
eventsa = postprocess.makeEventList(resultbina, 1, 10)
ICMEsa = postprocess.removeCreepy(eventsa, 3)
test_cloudsa = [x for x in evtLista if (x.begin.year in testa)]

# evaluate STEREO B

resultb.index = pds.to_datetime(resultb.index)
resultbinb = postprocess.make_binary(resultb['pred'], 0.5)
eventsb = postprocess.makeEventList(resultbinb, 1, 10)
ICMEsb = postprocess.removeCreepy(eventsb, 3)
test_cloudsb = [x for x in evtListb if (x.begin.year in testb)]

We can then plot the events and their associated similarity values to compare true label to predicted label.

In [15]:
# Score by event WIND

print('WIND:')

TPw, FNw, FPw, detectedw = postprocess.evaluate(ICMEsw, test_cloudsw, thres=0.1)
print('Precision is:',len(TPw)/(len(TPw)+len(FPw)))
print('Recall is:',len(TPw)/(len(TPw)+len(FNw)))
print('True Positives', len(TPw))
print('False Negatives', len(FNw))
print('False Positives', len(FPw))

# Score by event STEREO A

print('---------------------')

print('STEREO A:')

TPa, FNa, FPa, detecteda = postprocess.evaluate(ICMEsa, test_cloudsa, thres=0.1)
print('Precision is:',len(TPa)/(len(TPa)+len(FPa)))
print('Recall is:',len(TPa)/(len(TPa)+len(FNa)))
print('True Positives', len(TPa))
print('False Negatives', len(FNa))
print('False Positives', len(FPa))

# Score by event STEREO B

print('---------------------')

print('STEREO B:')

TPb, FNb, FPb, detectedb = postprocess.evaluate(ICMEsb, test_cloudsb, thres=0.1)
print('Precision is:',len(TPb)/(len(TPb)+len(FPb)))
print('Recall is:',len(TPb)/(len(TPb)+len(FNb)))
print('True Positives', len(TPb))
print('False Negatives', len(FNb))
print('False Positives', len(FPb))

WIND:
Precision is: 0.625
Recall is: 0.45454545454545453
True Positives 20
False Negatives 24
False Positives 12
---------------------
STEREO A:
Precision is: 0.48
Recall is: 0.6923076923076923
True Positives 36
False Negatives 16
False Positives 39
---------------------
STEREO B:
Precision is: 0.6
Recall is: 0.5675675675675675
True Positives 21
False Negatives 16
False Positives 14


In [16]:
startlagw = []
endlagw = []

for i in range(0, len(detectedw)):
    predstartw = TPw[i].begin
    predendw = TPw[i].end
    startlagw.append((predstartw-detectedw[i].begin).total_seconds()/60)
    endlagw.append((predendw-detectedw[i].end).total_seconds()/60)
    
startlaga = []
endlaga = []

for i in range(0, len(detecteda)):
    predstarta = TPa[i].begin
    predenda = TPa[i].end
    startlaga.append((predstarta-detecteda[i].begin).total_seconds()/60)
    endlaga.append((predenda-detecteda[i].end).total_seconds()/60)
    
startlagb = []
endlagb = []

for i in range(0, len(detectedb)):
    predstartb = TPb[i].begin
    predendb = TPb[i].end
    startlagb.append((predstartb-detectedb[i].begin).total_seconds()/60)
    endlagb.append((predendb-detectedb[i].end).total_seconds()/60)

In [17]:
import pickle

with open('startlag1'+str(split)+'.p', 'wb') as fp:
    pickle.dump(startlaga, fp)
    
with open('endlag1'+str(split)+'.p', 'wb') as fp:
    pickle.dump(endlaga, fp)

with open('startlag2'+str(split)+'.p', 'wb') as fp:
    pickle.dump(startlagb, fp)
    
with open('endlag2'+str(split)+'.p', 'wb') as fp:
    pickle.dump(endlagb, fp)
    
with open('startlag0'+str(split)+'.p', 'wb') as fp:
    pickle.dump(startlagw, fp)
    
with open('endlag0'+str(split)+'.p', 'wb') as fp:
    pickle.dump(endlagw, fp)

In [18]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
from cm import make_confusion_matrix

cmw = confusion_matrix(resultw['true'], resultbinw)
cma = confusion_matrix(resulta['true'], resultbina)
cmb = confusion_matrix(resultb['true'], resultbinb)

np.savetxt('cmw'+str(split)+'.txt',cmw)
cmeventw = np.array([[0,len(FPw)],[len(FNw),len(TPw)]])
np.savetxt('cmeventw'+str(split)+'.txt',cmeventw)

np.savetxt('cma'+str(split)+'.txt',cma)
cmeventa = np.array([[0,len(FPa)],[len(FNa),len(TPa)]])
np.savetxt('cmeventa'+str(split)+'.txt',cmeventa)

np.savetxt('cmb'+str(split)+'.txt',cmb)
cmeventb = np.array([[0,len(FPb)],[len(FNb),len(TPb)]])
np.savetxt('cmeventb'+str(split)+'.txt',cmeventb)